In [1]:
!pip install pandas
!pip install -U scikit-learn

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
import torch
from torchvision import datasets, transforms
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

from resnet import resnet50 as self_resnet50

import os
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 自定义数据集类
class MyDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.classes = os.listdir(root_dir)
        self.file_paths = []
        self.labels = []
        for i, class_name in enumerate(self.classes):
            class_path = os.path.join(root_dir, class_name)
            files = os.listdir(class_path)
            self.file_paths.extend([os.path.join(class_path, file) for file in files])
            self.labels.extend([i] * len(files))

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]
        data = np.loadtxt(file_path).reshape(32, 512)
        data = torch.from_numpy(data).float()
        return data, label
    
    def get_info(self, idx):
        """ 返回文件路径和标签 """
        return self.file_paths[idx], self.labels[idx]

# 主函数
if __name__ == "__main__":
    # 设置随机种子
    torch.manual_seed(42)

    # 数据集目录和文件路径
    data_dir = 'data_txt/'

    # 构建数据集
    dataset = MyDataset(data_dir)

    # 划分训练集和测试集
    train_indices, test_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
    train_dataset = torch.utils.data.Subset(dataset, train_indices)
    test_dataset = torch.utils.data.Subset(dataset, test_indices)

    # 创建数据加载器
    batch_size = 128
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    # 实例化ResNet50模型
    resnet50 = self_resnet50(num_classes=5)

    # 指定设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    resnet50 = resnet50.to(device)

    # 定义损失函数和优化器
    loss_fn = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(resnet50.parameters(), lr=0.01, momentum=0.9, nesterov=True)

    # 训练参数
    epoch = 1

    # 训练循环
    for i in range(epoch):
        print("-------第{}轮训练开始-------".format(i + 1))
        resnet50.train()
        for data in tqdm(train_dataloader, ncols=100, desc='Train'):
            imgs, labels = data
            imgs = imgs.to(device).reshape(-1, 1, 32, 512)
            labels = labels.to(device)
            outputs = resnet50(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("训练次数：{}, Loss: {:.4f}".format(i + 1, loss.item()))

    # 提取训练数据的特征
    train_features = []
    train_labels = []
    resnet50.eval()
    for data in tqdm(train_dataloader, ncols=100, desc='Extracting Train Features'):
        imgs, labels = data
        imgs = imgs.to(device).reshape(-1, 1, 32, 512)
        with torch.no_grad():
            features = resnet50.extract_features(imgs)  # 假设此方法存在
            features = features.view(features.size(0), -1)
        train_features.extend(features.cpu().numpy())
        train_labels.extend(labels.numpy())

    # 训练SVM分类器
    svm_classifier = SVC(kernel='linear')
    svm_classifier.fit(train_features, train_labels)

    # 提取测试数据的特征并进行分类
    test_features = []
    test_labels = []
    for data in tqdm(test_dataloader, ncols=100, desc='Extracting Test Features'):
        imgs, labels = data
        imgs = imgs.to(device).reshape(-1, 1, 32, 512)
        with torch.no_grad():
            features = resnet50.extract_features(imgs)
            features = features.view(features.size(0), -1)
        test_features.extend(features.cpu().numpy())
        test_labels.extend(labels.numpy())

    # 使用SVM分类器进行预测
    svm_predictions = svm_classifier.predict(test_features)

    # 计算准确率
    accuracy = accuracy_score(test_labels, svm_predictions)
    print(f"SVM Accuracy: {accuracy:.4f}")

    # 结果写入文件
    with open('result.txt', 'w') as f:
        f.write(f"SVM Accuracy: {accuracy:.4f}\n")


-------第1轮训练开始-------


Train: 100%|██████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.26it/s]


训练次数：1, Loss: 10.0944


Extracting Test Features: 100%|███████████████████████████████████████| 1/1 [00:00<00:00,  1.78it/s]

SVM Accuracy: 1.0000
